Import of the relevant libraries

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

Use of BeautifulSoup to retrieve all tables of the Wikipedia page

In [2]:
r= requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(r.text, 'html.parser')
tables = [
    [
        [td.get_text(strip=True) for td in tr.find_all('td')] 
        for tr in table.find_all('tr')
    ] 
    for table in soup.find_all('table')
]

1st table corresponding to the required table is selected

In [3]:
requiredtable = tables[0]

Table transformed into a pandas dataframe with renaming of columns and deletion of the 1st blank row

In [4]:
df = pd.DataFrame(requiredtable, columns=['Postcode', 'Borough', 'Neighborhood'])
df.drop(df.index[0],inplace=True)

Filtering of the dataframe to ignore cells with a borough that is Not assigned

In [5]:
df = df[df['Borough'] != 'Not assigned']

Combining Neighborhoods into one row that have the same Postcode & Borough 

In [6]:
df = df.sort_values(['Postcode','Borough','Neighborhood'])
df = df.groupby(['Postcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()

If a cell has a borough but a "Not assigned" neighborhood, then the neighborhood will be the same as the borough.

In [7]:
df.Neighborhood.replace("Not assigned",df.Borough,inplace=True)

Number of rows of the final dataframe

In [8]:
df.shape

(103, 3)